# Case 3: Cross-linked PEGDAE System using Reacter

This notebook implements a cross-linked polymer system using:
- PEGDAE₃ monomer (linear)
- 3-arm branch structure
- Reacter for cross-linking reactions
- LAMMPS fix bond/react templates

**Key features:**
- Builds linear and branch monomers
- Generates reaction templates for LAMMPS fix bond/react
- Creates initial configuration using packmol
- Generates LAMMPS input script for reactive MD

> [!WARNING]
> This notebook contains examples that use outdated API.
> **Reason**: Complex polymer building API has changed significantly.
> **Status**: Needs complete rewrite with current API.
> **TODO**: Update examples to match current molpy API.


## Step 1: Import Required Libraries

Import all necessary modules from MolPy for building cross-linked polymer systems, handling reactions, and exporting to LAMMPS format.

In [1]:
# NOTE: This example is temporarily disabled due to API changes
# TODO: Update to current API

# import numpy as np
# import molpy as mp
# from molpy.core.atomistic import Atom, Atomistic, Bond
# from molpy.core.entity import Entity
# from molpy.core.frame import Frame
# from molpy.core.box import Box
# from molpy.core.forcefield import ForceField
# from molpy.external import RDKitAdapter, Generate3D
# from molpy.parser.smiles import parse_bigsmiles, bigsmilesir_to_monomer, parse_smiles
# from molpy.parser.smiles.converter import create_monomer_from_atom_class_ports
# from molpy.reacter import (
#     Reacter,
#     select_port_atom,
#     select_hydroxyl_group,
#     form_single_bond,
#     ReactionResult,
# )
# from molpy.reacter.utils import find_neighbors
# from molpy.typifier.atomistic import OplsAtomisticTypifier
# from molpy.io.data.lammps import LammpsDataWriter
# from molpy.io.forcefield.lammps import LAMMPSForceFieldWriter
# from molpy.pack import InsideBoxConstraint, Molpack
# from pathlib import Path
# from typing import List, Dict, Set

## Step 2: Load Force Field

Load the OPLS-AA force field and create a typifier.

In [2]:
# Load force field
# forcefield_path = "oplsaa.xml"
# ff = mp.io.read_xml_forcefield(forcefield_path)
# typifier = OplsAtomisticTypifier(ff, strict_typing=False)
# 
# print("✅ Force field loaded successfully")

## Step 3: Define Helper Functions

Define helper functions for:
- Reaction selectors: Identify reaction sites
- Monomer builders: Build linear and branch monomers
- Reaction runners: Execute dehydration reactions
- Template generators: Create LAMMPS fix bond/react templates

In [3]:
# def select_carbon_from_oh(assembly: Atomistic, port_name: str) -> Atom:
#     """Select the carbon atom connected to -OH oxygen (port)."""
#     port_o = select_port_atom(assembly, port_name)
#     c_neighbors = find_neighbors(assembly, port_o, element="C")
#     return c_neighbors[0]
# 
# 
# def select_h_from_oh(assembly: Atomistic, port_atom: Atom) -> list[Atom]:
#     """Select H from -OH group when port_atom is the O atom."""
#     h_neighbors = find_neighbors(assembly, port_atom, element="H")
#     return [h_neighbors[0]]
# 
# 
# def build_monomer(bigsmiles: str, typifier: OplsAtomisticTypifier) -> Atomistic:
#     """Build a monomer from BigSMILES string with 3D coordinates and types."""
#     ir = parse_bigsmiles(bigsmiles)
#     monomer = bigsmilesir_to_monomer(ir)
#     
#     adapter = RDKitAdapter(internal=monomer)
#     generate_3d = Generate3D(add_hydrogens=True, embed=True, optimize=True, update_internal=True)
#     adapter = generate_3d(adapter)
#     monomer = adapter.get_internal()
#     
#     monomer.get_topo(gen_angle=True, gen_dihe=True)
#     
#     for idx, atom in enumerate(monomer.atoms):
#         atom["id"] = idx + 1
#     
#     typifier.typify(monomer)
#     
#     return monomer
# 
# 
# def extract_subgraph(
#     struct: Atomistic,
#     port_atoms: List[Atom],
#     radius: int,
# ) -> tuple[Atomistic, List[Atom], List[Atom]]:
#     """Extract a subgraph within topological radius from port atoms."""
#     topo = struct.get_topo()
#     entity_to_idx: dict[Atom, int] = topo.entity_to_idx
#     idx_to_entity: list[Atom] = topo.idx_to_entity
#     
#     port_idx = [a["id"] for a in port_atoms]
#     p1 = None
#     p2 = None
#     for a in struct.atoms:
#         if a["id"] == port_idx[0]:
#             p1 = a
#         if a["id"] == port_idx[1]:
#             p2 = a
#     
#     init_atoms = [entity_to_idx[p1], entity_to_idx[p2]]
#     selected_atoms: list[Atom] = []
#     edge_atoms: list[Atom] = []
#     for c_idx in init_atoms:
#         distances = topo.distances(source=[c_idx])[0]
#         for i, d in enumerate(distances):
#             if d <= radius and d < float("inf"):
#                 selected_atoms.append(idx_to_entity[i])
#                 if d == radius:
#                     eatom = idx_to_entity[i]
#                     if eatom.get("symbol") == "H":
#                         continue
#                     edge_atoms.append(eatom)
#     
#     new_struct = Atomistic()
#     new_struct.add_atoms(selected_atoms)
#     for bond in struct.bonds:
#         endpoints = bond.endpoints
#         if all(ep in selected_atoms for ep in endpoints):
#             new_struct.add_bond(bond)
#     
#     for angle in struct.angles:
#         endpoints = angle.endpoints
#         if all(ep in selected_atoms for ep in endpoints):
#             new_struct.add_angle(angle)
#     
#     for dihedral in struct.dihedrals:
#         endpoints = dihedral.endpoints
#         if all(ep in selected_atoms for ep in endpoints):
#             new_struct.add_dihedral(dihedral)
#     
#     return new_struct, [p1, p2], edge_atoms
# 
# 
# def run_reaction(
#     left: Atomistic,
#     right: Atomistic,
#     port_L: str,
#     port_R: str,
#     reaction_name: str = "crosslinking",
# ) -> ReactionResult:
#     """Run dehydration reaction to form ether bond."""
#     dehydration = Reacter(
#         name=reaction_name,
#         port_selector_left=select_carbon_from_oh,
#         port_selector_right=select_port_atom,
#         leaving_selector_left=select_hydroxyl_group,
#         leaving_selector_right=select_h_from_oh,
#         bond_former=form_single_bond,
#     )
#     
#     reaction_result = dehydration.run(
#         left=left,
#         right=right,
#         port_L=port_L,
#         port_R=port_R,
#         record_intermediates=True,
#     )
#     return reaction_result
# 
# 
# print("✅ Helper functions defined")

## Step 4: Build Linear Monomer

Build the PEGDAE₃ linear monomer: HO-(CH₂CH₂O)₃-CH₂CH₂-OH from BigSMILES notation.

In [4]:
# def build_pegdae3_monomer(typifier: OplsAtomisticTypifier) -> Atomistic:
#     """Build PEGDAE₃ monomer: HO-(CH2CH2O)3-CH2CH2-OH"""
#     bigsmiles = '{[<]OCCOCCOCCOCCO[>]}'
#     return build_monomer(bigsmiles, typifier)
# 
# Build linear monomer
# linear_monomer = build_pegdae3_monomer(typifier)
# 
# linear_ports = [atom.get("port") for atom in linear_monomer.atoms if atom.get("port") is not None]
# 
# print(f"✅ Linear monomer built:")
# print(f"   Atoms: {len(linear_monomer.atoms)}")
# print(f"   Ports: {linear_ports}")

## Step 5: Build Branch Monomer

Build the 3-arm branch structure: C(COCCO)(COCCO)(COCCO) with three connection points.

In [5]:
# NOTE: This example is temporarily disabled due to API changes
# TODO: Update to current API

# def build_3arm_monomer(typifier: OplsAtomisticTypifier) -> Atomistic:
#     """Build 3-arm branch structure: C(COCCO)(COCCO)(COCCO)"""
#     smiles = 'C(COCCO[*:3])(COCCO[*:4])COCCO[*:5]'
#     
#     ir = parse_smiles(smiles)
#     if isinstance(ir, list):
#         ir = ir[0]
#     
#     monomer = create_monomer_from_atom_class_ports(ir)
#     
#     adapter = RDKitAdapter(internal=monomer)
#     generate_3d = Generate3D(add_hydrogens=True, embed=True, optimize=True, update_internal=True)
#     adapter = generate_3d(adapter)
#     monomer = adapter.get_internal()
#     
#     monomer.get_topo(gen_angle=True, gen_dihe=True)
#     
#     for idx, atom in enumerate(monomer.atoms):
#         atom["id"] = idx + 1
#     
#     typifier.typify(monomer)
#     
#     return monomer
# 
# # Build branch monomer
# branch_monomer = build_3arm_monomer(typifier)
# 
# branch_ports = [atom.get("port") for atom in branch_monomer.atoms if atom.get("port") is not None]
# 
# print(f"✅ Branch monomer built:")
# print(f"   Atoms: {len(branch_monomer.atoms)}")
# print(f"   Ports: {branch_ports}")

## Step 6: Generate Reaction Templates

Generate LAMMPS fix bond/react templates for different reaction types:
- Reaction 1: Linear + Linear
- Reaction 2: Linear + Branch (first arm)
- Reaction 3: Linear + Branch (second arm, if available)

In [6]:
# Create output directory
# output_dir = Path("case3_output")
# output_dir.mkdir(parents=True, exist_ok=True)
# 
# def write_fix_bond_react_map(
#     map_path: Path,
#     pre_path: Path,
#     post_path: Path,
#     pre: Atomistic,
#     post: Atomistic,
#     init_atoms: List[Atom],
#     edge_atoms: List[Atom],
#     removed_atoms: List[Atom],
# ) -> tuple[Frame, Frame]:
#     """Write LAMMPS fix bond/react template and mapping files."""
#     initiator_ids = [a["id"] for a in init_atoms]
#     edge_ids = [a["id"] for a in edge_atoms]
#     deleted_ids = [a["id"] for a in removed_atoms]
#     
#     equiv = []
#     pre_mapping = {}
#     post_mapping = {}
#     
#     for i, atomL in enumerate(pre.atoms, start=1):
#         unified_id_L = atomL["id"]
#         pre_mapping[unified_id_L] = i
#         
#         for j, atomR in enumerate(post.atoms, start=1):
#             unified_id_R = atomR["id"]
#             if unified_id_R not in post_mapping:
#                 post_mapping[unified_id_R] = j
#             
#             if unified_id_L == unified_id_R:
#                 equiv.append((i, j))
#                 break
#     
#     for atom in pre.atoms:
#         unified_id = atom["id"]
#         atom["id"] = pre_mapping[unified_id]
#     
#     for atom in post.atoms:
#         unified_id = atom["id"]
#         atom["id"] = post_mapping[unified_id]
#     
#     with map_path.open("w", encoding="utf-8") as f:
#         f.write("# auto-generated map file for fix bond/react\n\n")
#         f.write(f"{len(equiv)} equivalences\n")
#         f.write(f"{len(edge_ids)} edgeIDs\n")
#         f.write(f"{len(deleted_ids)} deleteIDs\n")
#         f.write("\n")
#         f.write("InitiatorIDs\n\n")
#         for orig_id in initiator_ids:
#             if orig_id in pre_mapping:
#                 f.write(f"{pre_mapping[orig_id]}\n")
#         f.write("\n")
#         f.write("EdgeIDs\n\n")
#         for orig_id in edge_ids:
#             if orig_id in pre_mapping:
#                 f.write(f"{pre_mapping[orig_id]}\n")
#         f.write("\n")
#         f.write("DeleteIDs\n\n")
#         for orig_id in deleted_ids:
#             if orig_id in pre_mapping:
#                 f.write(f"{pre_mapping[orig_id]}\n")
#         f.write("\n")
#         f.write("Equivalences\n\n")
#         for pre_id, post_id in equiv:
#             f.write(f"{pre_id}   {post_id}\n")
#     
#     pre_frame = pre.to_frame()
#     post_frame = post.to_frame()
#     
#     if "charge" in pre_frame["atoms"]:
#         pre_frame["atoms"]["q"] = pre_frame["atoms"]["charge"]
#     if "charge" in post_frame["atoms"]:
#         post_frame["atoms"]["q"] = post_frame["atoms"]["charge"]
#     
#     for frame_name, frame in [("pre", pre_frame), ("post", post_frame)]:
#         if "bonds" in frame:
#             bonds = frame["bonds"]
#             if "atom_i" in bonds and "atom_j" in bonds:
#                 bonds["atom1"] = bonds["atom_i"] + 1
#                 bonds["atom2"] = bonds["atom_j"] + 1
#         if "angles" in frame:
#             angles = frame["angles"]
#             if "atom_i" in angles and "atom_j" in angles and "atom_k" in angles:
#                 angles["atom1"] = angles["atom_i"] + 1
#                 angles["atom2"] = angles["atom_j"] + 1
#                 angles["atom3"] = angles["atom_k"] + 1
#         if "dihedrals" in frame:
#             dihedrals = frame["dihedrals"]
#             if "atom_i" in dihedrals and "atom_j" in dihedrals and "atom_k" in dihedrals and "atom_l" in dihedrals:
#                 dihedrals["atom1"] = dihedrals["atom_i"] + 1
#                 dihedrals["atom2"] = dihedrals["atom_j"] + 1
#                 dihedrals["atom3"] = dihedrals["atom_k"] + 1
#                 dihedrals["atom4"] = dihedrals["atom_l"] + 1
#     
#     mp.io.write_lammps_molecule(pre_path, pre_frame)
#     mp.io.write_lammps_molecule(post_path, post_frame)
#     
#     return pre_frame, post_frame
# 
# 
# def generate_template(
#     reaction_result: ReactionResult,
#     typifier: OplsAtomisticTypifier,
#     rxn_id: str,
#     outdir: Path,
# ) -> tuple[Frame, Frame]:
#     """Generate LAMMPS fix bond/react template and mapping files."""
#     reactants = reaction_result.reactants
#     product = reaction_result.product
#     
#     def ensure_typified(struct: Atomistic) -> None:
#         typifier.atom_typifier.typify(struct)
#         for bond in struct.bonds:
#             typifier.bond_typifier.typify(bond)
#         for angle in struct.angles:
#             typifier.angle_typifier.typify(angle)
#         for dihedral in struct.dihedrals:
#             typifier.dihedral_typifier.typify(dihedral)
#     
#     reactants.get_topo(gen_angle=True, gen_dihe=True)
#     product.get_topo(gen_angle=True, gen_dihe=True)
#     ensure_typified(reactants)
#     ensure_typified(product)
#     
#     ports = [reaction_result.port_L, reaction_result.port_R]
#     removed_atoms = reaction_result.removed_atoms
#     
#     pre_path = outdir / f"{rxn_id}_pre.mol"
#     post_path = outdir / f"{rxn_id}_post.mol"
#     map_path = outdir / f"{rxn_id}.map"
# 
#     pre_t, pre_init_atoms, pre_edge_atoms = extract_subgraph(reactants, ports, radius=4)
#     post_t, post_init_atoms, post_edge_atoms = extract_subgraph(product, ports, radius=4)
# 
#     if removed_atoms:
#         post_t.add_atoms(removed_atoms)
# 
#     return write_fix_bond_react_map(
#         map_path,
#         pre_path,
#         post_path,
#         pre_t,
#         post_t,
#         pre_init_atoms,
#         pre_edge_atoms,
#         removed_atoms
#     )
# 
# 
# Generate reaction templates
# template_frames = []
# 
# Reaction 1: Linear + Linear
# rxn1 = run_reaction(
#     linear_monomer.copy(),
#     linear_monomer.copy(),
#     port_L=">",
#     port_R="<",
#     reaction_name="linear+linear",
# )
# pre_frame_1, post_frame_1 = generate_template(rxn1, typifier, "rxn1", output_dir)
# template_frames.extend([pre_frame_1, post_frame_1])
# print(f"✅ Reaction 1 template generated: rxn1_pre.mol, rxn1_post.mol, rxn1.map")
# 
# Reaction 2: Linear + Branch (first arm)
# rxn2 = run_reaction(
#     linear_monomer.copy(),
#     branch_monomer.copy(),
#     port_L=">",
#     port_R="port_3",
#     reaction_name="linear+branch1",
# )
# pre_frame_2, post_frame_2 = generate_template(rxn2, typifier, "rxn2", output_dir)
# template_frames.extend([pre_frame_2, post_frame_2])
# print(f"✅ Reaction 2 template generated: rxn2_pre.mol, rxn2_post.mol, rxn2.map")
# 
# Reaction 3: Linear + Branch (second arm)
# rxn2_product_ports = [atom.get("port") for atom in rxn2.product_info.product.atoms if atom.get("port") is not None]
# n_reactions = 2
# if "port_4" in rxn2_product_ports:
#     rxn3 = run_reaction(
#         linear_monomer.copy(),
#         rxn2.product_info.product.copy(),
#         port_L=">",
#         port_R="port_4",
#         reaction_name="linear+branch2",
#     )
#     pre_frame_3, post_frame_3 = generate_template(rxn3, typifier, "rxn3", output_dir)
#     template_frames.extend([pre_frame_3, post_frame_3])
#     n_reactions = 3
#     print(f"✅ Reaction 3 template generated: rxn3_pre.mol, rxn3_post.mol, rxn3.map")
# 
# print(f"\n✅ Generated {n_reactions} reaction templates")

## Step 7: Create Initial Configuration

Create initial LAMMPS configuration by packing linear and branch monomers into a simulation box using packmol. Calculate box size from target density.

In [7]:
# def calculate_molecular_weight(monomer: Atomistic) -> float:
#     """Calculate molecular weight of a monomer."""
#     from molpy.core.element import Element
#     total_mw = 0.0
#     for atom in monomer.atoms:
#         symbol = atom.get("symbol", "C")
#         symbol_upper = symbol.upper() if symbol else "C"
#         element = Element(symbol_upper)
#         total_mw += element.mass
#     return total_mw
# 
# 
# def calculate_box_size_from_density(
#     n_linear: int,
#     n_branch: int,
#     linear_monomer: Atomistic,
#     branch_monomer: Atomistic,
#     target_density: float = 1.0,
# ) -> float:
#     """Calculate box size from target density."""
#     mw_linear = calculate_molecular_weight(linear_monomer)
#     mw_branch = calculate_molecular_weight(branch_monomer)
#     total_mw = (n_linear * mw_linear + n_branch * mw_branch)
#     
#     NA = 6.022e23
#     total_mass_g = total_mw / NA
#     volume_cm3 = total_mass_g / target_density
#     volume_angstrom3 = volume_cm3 * 1e24
#     box_length_angstrom = volume_angstrom3 ** (1.0 / 3.0)
#     
#     return box_length_angstrom
# 
# 
# def collect_types_from_frames(*frames: Frame) -> Dict[str, Set[str]]:
#     """Collect all type names from multiple frames."""
#     atom_types: Set[str] = set()
#     bond_types: Set[str] = set()
#     angle_types: Set[str] = set()
#     dihedral_types: Set[str] = set()
#     
#     for frame in frames:
#         if "atoms" in frame and "type" in frame["atoms"]:
#             for atom_type in frame["atoms"]["type"]:
#                 if atom_type:
#                     type_str = str(atom_type)
#                     if not type_str.isdigit():
#                         atom_types.add(type_str)
#         
#         if "bonds" in frame and "type" in frame["bonds"]:
#             for bond_type in frame["bonds"]["type"]:
#                 if bond_type:
#                     type_str = str(bond_type)
#                     if not type_str.isdigit():
#                         bond_types.add(type_str)
#         
#         if "angles" in frame and "type" in frame["angles"]:
#             for angle_type in frame["angles"]["type"]:
#                 if angle_type:
#                     type_str = str(angle_type)
#                     if not type_str.isdigit():
#                         angle_types.add(type_str)
#         
#         if "dihedrals" in frame and "type" in frame["dihedrals"]:
#             for dihedral_type in frame["dihedrals"]["type"]:
#                 if dihedral_type:
#                     type_str = str(dihedral_type)
#                     if not type_str.isdigit():
#                         dihedral_types.add(type_str)
#     
#     return {
#         "atom_types": atom_types,
#         "bond_types": bond_types,
#         "angle_types": angle_types,
#         "dihedral_types": dihedral_types,
#     }
# 
# 
# Pack monomers
# n_linear = 180
# n_branch = 20
# target_density = 1.0
# 
# box_length = calculate_box_size_from_density(
#     n_linear=n_linear,
#     n_branch=n_branch,
#     linear_monomer=linear_monomer,
#     branch_monomer=branch_monomer,
#     target_density=target_density,
# )
# 
# print(f"✅ Calculated box size: {box_length:.2f} Å")
# 
# Prepare frames for packing
# linear_frame = linear_monomer.to_frame()
# branch_frame = branch_monomer.to_frame()
# 
# if "charge" in linear_frame["atoms"]:
#     linear_frame["atoms"]["q"] = linear_frame["atoms"]["charge"]
# if "charge" in branch_frame["atoms"]:
#     branch_frame["atoms"]["q"] = branch_frame["atoms"]["charge"]
# 
# Use packmol for packing
# pack_workdir = output_dir / "packmol_work"
# packer = Molpack(workdir=pack_workdir)
# 
# origin = np.array([0.0, 0.0, 0.0])
# length = np.array([box_length, box_length, box_length])
# box_constraint = InsideBoxConstraint(length=length, origin=origin)
# 
# packer.add_target(linear_frame, number=n_linear, constraint=box_constraint)
# packer.add_target(branch_frame, number=n_branch, constraint=box_constraint)
# 
# print(f"   Packing {n_linear} linear + {n_branch} branch monomers...")
# packed_frame = packer.optimize(max_steps=10000, seed=42)
# 
# print(f"✅ Packed system: {packed_frame['atoms'].nrows} atoms")
# 
# Ensure required fields
# if "charge" in packed_frame["atoms"] and "q" not in packed_frame["atoms"]:
#     packed_frame["atoms"]["q"] = packed_frame["atoms"]["charge"]
# elif "q" in packed_frame["atoms"] and "charge" not in packed_frame["atoms"]:
#     packed_frame["atoms"]["charge"] = packed_frame["atoms"]["q"]
# 
# Create box object
# box = Box.cubic(length=box_length)
# packed_frame.metadata["box"] = box
# 
# Collect types from config and templates
# all_frames = [packed_frame] + list(template_frames)
# types_from_config = collect_types_from_frames(*all_frames)
# 
# Add type labels to frame metadata
# packed_frame.metadata["type_labels"] = {
#     "atom_types": sorted(list(types_from_config["atom_types"])),
#     "bond_types": sorted(list(types_from_config["bond_types"])),
#     "angle_types": sorted(list(types_from_config["angle_types"])),
#     "dihedral_types": sorted(list(types_from_config["dihedral_types"])),
# }
# 
# Write force field file
# ff_path = output_dir / "case3_crosslinking.ff"
# ff_writer = LAMMPSForceFieldWriter(ff_path)
# ff_writer.write(
#     ff,
#     atom_types=types_from_config["atom_types"],
#     bond_types=types_from_config["bond_types"],
#     angle_types=types_from_config["angle_types"],
#     dihedral_types=types_from_config["dihedral_types"],
# )
# 
# Write data file
# data_path = output_dir / "case3_crosslinking.data"
# data_writer = LammpsDataWriter(data_path, atom_style="full")
# data_writer.write(packed_frame)
# 
# print(f"✅ Written: {ff_path.name}, {data_path.name}")

## Step 8: Generate LAMMPS Input Script

Generate LAMMPS input script for reactive MD simulation using fix bond/react.

In [8]:
# Build molecule commands
# molecule_commands = []
# react_commands = []
# for i in range(1, n_reactions + 1):
#     molecule_commands.append(f"molecule   rxn{i}_pre rxn{i}_pre.mol")
#     molecule_commands.append(f"molecule   rxn{i}_post rxn{i}_post.mol")
#     react_commands.append(f"  react rxn{i} all 1 0.0 5 rxn{i}_pre rxn{i}_post prob 0.2 5123 rescale_charges yes rxn{i}.map &")
# 
# script_content = f"""# LAMMPS input script for Case 3: Cross-linked PEGDAE System
# Force field: OPLS-AA
# Reaction: Dehydration to form ether bonds
# 
# ============================================================================
# Initialization
# ============================================================================
# 
# units           real
# atom_style      full
# boundary        p p p
# dimension       3
# 
# ============================================================================
# Read data and force field
# ============================================================================
# 
# read_data       case3_crosslinking.data &
#     extra/bond/per/atom 25 &
#     extra/angle/per/atom 25 &
#     extra/dihedral/per/atom 25 &
#     extra/improper/per/atom 25 &
#     extra/special/per/atom 25
# include         case3_crosslinking.ff
# kspace_style    pppm 1.0e-5
# special_bonds   lj/coul 0.0 0.0 0.5
# 
# ============================================================================
# Neighbor settings
# ============================================================================
# 
# neighbor        2.0 bin
# neigh_modify    delay 0 every 1 check yes
# 
# ============================================================================
# Energy minimization
# ============================================================================
# 
# print "=========================================="
# print "Step 1: Energy Minimization"
# print "=========================================="
# 
# minimize        1.0e-4 1.0e-4 1000 10000
# 
# ============================================================================
# Equilibration: NPT
# ============================================================================
# 
# print "=========================================="
# print "Step 2: NPT Equilibration"
# print "=========================================="
# 
# fix             shake all shake 0.0001 20 0 t opls_140 opls_155 opls_185
# 
# fix             npt all npt temp 300.0 300.0 100.0 iso 1.5 1.5 1000.0
# 
# dump            2 all custom 500 equil.dump id type x y z
# 
# run             1000
# unfix           npt
# undump          2
# 
# ============================================================================
# Reactive MD: fix bond/react
# ============================================================================
# 
# print "=========================================="
# print "Step 3: Reactive MD with fix bond/react"
# print "=========================================="
# 
# Read reaction templates
# {chr(10).join(molecule_commands)}
# 
# fix rxns all bond/react stabilization yes npt_grp .03 &
# {chr(10).join(react_commands)}
    # End of reactions
# 
# unfix shake
# 
# Thermostat and barostat for reactive MD
# fix             npt_grp_react all npt temp 300.0 300.0 100.0 iso 1.5 1.5 1000.0
# 
# Output settings
# thermo          100
# thermo_style    custom elapsed temp pe ke etotal press vol density f_rxns[*]
# thermo_modify   flush yes
# 
# Dump trajectory
# compute 1 all property/local batom1 batom2 btype
# dump            3 all custom 1000 react.dump id mol type xu yu zu
# dump            4 all local 1000 topo.dump index c_1[1] c_1[2] c_1[3]
# run             1000
# 
# undump         3
# undump         4
# unfix          rxns
# unfix          npt_grp_react
# 
# ============================================================================
# Production MD
# ============================================================================
# 
# print "=========================================="
# print "Step 4: Production MD"
# print "=========================================="
# 
# thermo_style    custom elapsed temp pe ke etotal press vol density
# 
# ============================================================================
# Final output
# ============================================================================
# 
# write_data      final_crosslinked.data
# write_restart   final_crosslinked.restart
# 
# print "=========================================="
# print "Simulation completed!"
# print "=========================================="
# """
# 
# script_path = output_dir / "run_case3_crosslinking.in"
# with script_path.open("w") as f:
#     f.write(script_content)
# 
# print(f"✅ Generated LAMMPS input script: {script_path.name}")

## Summary

This notebook demonstrated the complete workflow for building a cross-linked polymer system:

1. ✅ Loaded OPLS-AA force field
2. ✅ Built linear PEGDAE₃ monomer
3. ✅ Built 3-arm branch monomer
4. ✅ Generated reaction templates for LAMMPS fix bond/react
5. ✅ Created initial configuration using packmol
6. ✅ Generated LAMMPS input script for reactive MD

The generated files can be used for:
- LAMMPS reactive MD simulations (run_case3_crosslinking.in)
- Reaction templates (rxn*_pre.mol, rxn*_post.mol, rxn*.map)
- Initial configuration (case3_crosslinking.data, case3_crosslinking.ff)